In [1]:
import requests
import pandas as pd
import numpy as np

import yfinance as yf
from fredapi import Fred

fred_apikey = '4d68b01a5415707192c45e61f180299d'
fred_api_endpoint = 'https://api.stlouisfed.org/fred/series/observations'

fred_series_codes = [
    'GDPC1', # Real Gross Domestic Product
    'DFF', # Effective Federal Funds Rate (avg of daily rates)
    'FEDFUNDS', # Effective Federal Funds Rate (set by the fed)
    'BOPGSTB', # Balance of Payments
    'DGS1', # 1-Year Treasury Bond Yield
    'DGS2', # 2-Year Treasury Bond Yield
    'DGS3', # 3-Year Treasury Bond Yield
    'DGS5', # 5-Year Treasury Bond Yield
    'DGS7', # 7-Year Treasury Bond Yield
    'DGS10', # 10-Year Treasury Bond Yield
    'DGS20', # 20-Year Treasury Bond Yield
    'DGS30', # 30-Year Treasury Bond Yield
    'UNRATE', # Unemployment Rate
    'PAYEMS', # All Employees: Total Nonfarm Payrolls
    'PPIACO', # All Commodities: All Items PPI
    'CPALTT01USM659N', # All Commodities: All Items CPI
    'CORESTICKM159SFRBATL', # Core Personal Consumption Expenditures Price Index
    'UMCSENT', # Consumer Sentiment
    'PCECC96', # Personal Consumption Expenditures: Chain-type Price Index
    'PCEPI', # Personal Consumption Expenditures: Price Index
    'HOUST', # Total New Private Housing Units Started
    'HNFSUSNSA', # New One Family Houses for Sale
    'RHORUSQ156N', # Homeownership Rate
    'TOTALSA', # Total Vehicle Sales
    'RETAILMPCSMSA', # Total retail sales
    'RSXFS', # Retail Sales: Food Services and Drinking Places
    'WILL5000PR', # Wilshire 5000 Price Index
]

yfinance_tickers = [
    'ES=F', # S&P 500
    'NQ=F', # Nasdaq 100
    'YM=F', # Dow Jones Industrial Average
    'ZB=F', # 30-Year Treasury Bond
    'ZN=F', # 10-Year Treasury Bond
    'ZF=F', # 5-Year Treasury Bond
    'ZT=F', # 2-Year Treasury Bond
    'GC=F', # Gold
    'SI=F', # Silver
    'HG=F', # Copper
    'CL=F', # Crude Oil
    'HO=F', # Heating Oil
    'NG=F', # Natural Gas
    'RB=F', # RBOB Gasoline
    'ZC=F', # Corn
    'ZO=F', # Oats
    'KE=F', # KC HRW Wheat
    'ZR=F', # Rough Rice
    'ZS=F', # Soybeans
    'ZM=F', # Soybean Meal
    'HE=F', # Lean Hogs
    'LE=F', # Live Cattle
    'CC=F', # Cocoa
    'KC=F', # Coffee
    'CT=F', # Cotton
    'LBS=F', # Lumber
    'SB=F', # Sugar
    '^VIX', # CBOE Volatility Index
]

In [12]:
import datetime as dt

start = '2003-01-01'
end = dt.datetime.now()

yfinance_data = pd.DataFrame()
fred_data = pd.DataFrame()
for ticker in yfinance_tickers:
    yfinance_data[ticker] = yf.download(ticker, start=start, end=end)['Close']
futures_data = yfinance_data.interpolate(method='linear', limit_direction ='forward')
for series_id in fred_series_codes:
    params = {
        'series_id': series_id,
        'api_key': fred_apikey,
        'file_type': 'json',
        'limit': 100000
    }
    response = requests.get(fred_api_endpoint, params=params)
    data = pd.DataFrame(response.json()['observations']).drop(columns=['realtime_start', 'realtime_end']).set_index('date')
    data = data.set_index(pd.to_datetime(data.index))
    data = data.rename(columns={'value': series_id})
    fred_data = pd.concat([fred_data, data], axis=1)
fred_data = fred_data.ffill()

df = pd.concat([futures_data, fred_data], axis=1)
df = df.loc[start:end]
df = df.apply(pd.to_numeric, errors='coerce')
df = df.dropna()
df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,ES=F,NQ=F,YM=F,ZB=F,ZN=F,ZF=F,ZT=F,GC=F,SI=F,HG=F,...,UMCSENT,PCECC96,PCEPI,HOUST,HNFSUSNSA,RHORUSQ156N,TOTALSA,RETAILMPCSMSA,RSXFS,WILL5000PR
2003-01-02,907.50,1025.00,8591.0,110.156303,113.406303,112.156303,107.210899,346.100006,4.812000,0.7140,...,82.4,9232.782,81.664,1853.0,347.0,68.0,16.708,0.5,267230.0,8601.24
2003-01-03,910.00,1036.00,8604.0,110.281303,113.437500,112.156303,107.226601,351.200012,4.892000,0.7265,...,82.4,9232.782,81.664,1853.0,347.0,68.0,16.708,0.5,267230.0,8593.22
2003-01-06,926.50,1061.00,8737.0,110.031303,113.234398,111.937500,107.171898,351.700012,4.899000,0.7345,...,82.4,9232.782,81.664,1853.0,347.0,68.0,16.708,0.5,267230.0,8772.22
2003-01-07,923.75,1075.00,8750.0,110.343803,113.531303,112.187500,107.289101,347.299988,4.804000,0.7315,...,82.4,9232.782,81.664,1853.0,347.0,68.0,16.708,0.5,267230.0,8713.70
2003-01-08,910.00,1045.00,8583.0,111.031303,113.828102,112.359398,107.421898,353.899994,4.856000,0.7420,...,82.4,9232.782,81.664,1853.0,347.0,68.0,16.708,0.5,267230.0,8596.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-03,4049.75,12311.25,33414.0,123.843750,110.656250,106.328125,101.589844,1847.699951,21.093000,4.0780,...,64.9,14226.812,125.899,1309.0,436.0,65.9,15.336,-1.3,601529.0,41270.69
2023-03-06,4052.50,12323.50,33451.0,123.593750,110.562500,106.242188,101.519531,1847.900024,20.983999,4.1005,...,64.9,14226.812,125.899,1309.0,436.0,65.9,15.336,-1.3,601529.0,41226.53
2023-03-07,3989.75,12169.00,32864.0,124.000000,110.515625,106.023438,101.304688,1813.900024,20.054001,3.9865,...,64.9,14226.812,125.899,1309.0,436.0,65.9,15.336,-1.3,601529.0,40619.45
2023-03-08,3995.00,12228.25,32813.0,124.187500,110.453125,105.890625,101.199219,1812.699951,20.004999,4.0415,...,64.9,14226.812,125.899,1309.0,436.0,65.9,15.336,-1.3,601529.0,40670.72


In [13]:
# df to csv
df.to_csv('macro_data.csv')